# Análisis del riesgo de incumplimiento de los prestatarios

Preparar un informe para la división de préstamos de un banco. Deberás averiguar si el estado civil y el número de hijos de un cliente tienen un impacto en el incumplimiento de pago de un préstamo. El banco ya tiene algunos datos sobre la solvencia crediticia de los clientes.

El informe tendrá en cuenta al crear una **puntuación de crédito** para un cliente potencial. La **puntuación de crédito** se utiliza para evaluar la capacidad de un prestatario potencial para pagar su préstamo.

## Propósito del proyecto

- Evaluar el status quo de los clientes de un banco para encontrar clientes potenciales a recibir préstamos por parte del banco.
- Clasificar a los clientes potenciales a recibir un préstamo bancario como `candidato`, `probable candidato` y `no es candidato` con respecto a la probabilidad que tienen de pagarlo.

## Hipótesis

- Evaluar si el estado civil de un cliente es estadísticamente significativo en el incumplimiento de pago de un préstamo.
- Evaluar si el número de hijos de un cliente es estadísticamente significativo en el incumplimiento de pago de un préstamo.
- Evaluar si el propósito de préstamos de un cliente es estadísticamente significativo en el incumplimiento de pago de un préstamo.

## Información general.

In [2]:
import pandas as pd

try:
    data_bank = pd.read_csv('moved_credit_scoring_eng.csv')
except:
    data_bank = pd.read_csv('/datasets/credit_scoring_eng.csv')

In [3]:
data_bank.head(20)

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
0,1,-8437.673028,42,bachelor's degree,0,married,0,F,employee,0,40620.102,purchase of the house
1,1,-4024.803754,36,secondary education,1,married,0,F,employee,0,17932.802,car purchase
2,0,-5623.422610,33,Secondary Education,1,married,0,M,employee,0,23341.752,purchase of the house
3,3,-4124.747207,32,secondary education,1,married,0,M,employee,0,42820.568,supplementary education
4,0,340266.072047,53,secondary education,1,civil partnership,1,F,retiree,0,25378.572,to have a wedding
5,0,-926.185831,27,bachelor's degree,0,civil partnership,1,M,business,0,40922.170,purchase of the house
6,0,-2879.202052,43,bachelor's degree,0,married,0,F,business,0,38484.156,housing transactions
7,0,-152.779569,50,SECONDARY EDUCATION,1,married,0,M,employee,0,21731.829,education
8,2,-6929.865299,35,BACHELOR'S DEGREE,0,civil partnership,1,F,employee,0,15337.093,having a wedding
9,0,-2188.756445,41,secondary education,1,married,0,M,employee,0,23108.150,purchase of the house for my family


## Exploración de datos

**Descripción de los datos**
- `children` - el número de hijos en la familia
- `days_employed` - experiencia laboral en días
- `dob_years` - la edad del cliente en años
- `education` - la educación del cliente
- `education_id` - identificador de educación
- `family_status` - estado civil
- `family_status_id` - identificador de estado civil
- `gender` - género del cliente
- `income_type` - tipo de empleo
- `debt` - ¿había alguna deuda en el pago de un préstamo?
- `total_income` - ingreso mensual
- `purpose` - el propósito de obtener un préstamo

In [4]:
# Vamos a ver cuántas filas y columnas tiene nuestro conjunto de datos
rows = len(data_bank.axes[0])
cols = len(data_bank.axes[1])
print(f'Data Frame tiene {rows} filas y {cols} columnas')

Data Frame tiene 21525 filas y 12 columnas


In [5]:
# vamos a mostrar las primeras filas N
data_bank.head(10)

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
0,1,-8437.673028,42,bachelor's degree,0,married,0,F,employee,0,40620.102,purchase of the house
1,1,-4024.803754,36,secondary education,1,married,0,F,employee,0,17932.802,car purchase
2,0,-5623.422610,33,Secondary Education,1,married,0,M,employee,0,23341.752,purchase of the house
3,3,-4124.747207,32,secondary education,1,married,0,M,employee,0,42820.568,supplementary education
4,0,340266.072047,53,secondary education,1,civil partnership,1,F,retiree,0,25378.572,to have a wedding
5,0,-926.185831,27,bachelor's degree,0,civil partnership,1,M,business,0,40922.170,purchase of the house
6,0,-2879.202052,43,bachelor's degree,0,married,0,F,business,0,38484.156,housing transactions
7,0,-152.779569,50,SECONDARY EDUCATION,1,married,0,M,employee,0,21731.829,education
8,2,-6929.865299,35,BACHELOR'S DEGREE,0,civil partnership,1,F,employee,0,15337.093,having a wedding
9,0,-2188.756445,41,secondary education,1,married,0,M,employee,0,23108.150,purchase of the house for my family


Encontramos que `days_employed` es un tipo de datos coma flotante, debería de ser `int64`, por otra parte, las columnas de `days_employed` y `total_income` muestran valores ausentes.

In [6]:
# Obtener información sobre los datos
data_bank.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21525 entries, 0 to 21524
Data columns (total 12 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   children          21525 non-null  int64  
 1   days_employed     19351 non-null  float64
 2   dob_years         21525 non-null  int64  
 3   education         21525 non-null  object 
 4   education_id      21525 non-null  int64  
 5   family_status     21525 non-null  object 
 6   family_status_id  21525 non-null  int64  
 7   gender            21525 non-null  object 
 8   income_type       21525 non-null  object 
 9   debt              21525 non-null  int64  
 10  total_income      19351 non-null  float64
 11  purpose           21525 non-null  object 
dtypes: float64(2), int64(5), object(5)
memory usage: 2.0+ MB


Encontramos valores ausentes en las columnas `days_employed` y `total_income`, a primera vista, sin embargo, se puede hacer el razonamiento que los valores que faltan es debido a **clientes desempleado o que nunca han tarbajado**  por lo que debemos de confirmar que estos "datos ausentes" sea debido a estos eventos.

In [7]:
# Veamos la tabla filtrada con valores ausentes de la primera columna donde faltan datos
data_bank_filtered = data_bank[data_bank['days_employed'].isna()]
data_bank_filtered.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 2174 entries, 12 to 21510
Data columns (total 12 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   children          2174 non-null   int64  
 1   days_employed     0 non-null      float64
 2   dob_years         2174 non-null   int64  
 3   education         2174 non-null   object 
 4   education_id      2174 non-null   int64  
 5   family_status     2174 non-null   object 
 6   family_status_id  2174 non-null   int64  
 7   gender            2174 non-null   object 
 8   income_type       2174 non-null   object 
 9   debt              2174 non-null   int64  
 10  total_income      0 non-null      float64
 11  purpose           2174 non-null   object 
dtypes: float64(2), int64(5), object(5)
memory usage: 220.8+ KB


Se observa que los valores ausentes son simétricos con la cantidad de filas que hacen falta por lo que se puede inferir que no existen más valores ausentes más allá de los valores relacionados entre las columnas `days_employed` y `total_income`. La primera hipótesis es que los clientes que no trabajan no recibieron ingresos y, por ende, no hay valores en estas columnas.

In [8]:
# Apliquemos múltiples condiciones para filtrar datos y veamos el número de filas en la tabla filtrada.

data_bank_filtered.loc[:,['days_employed','total_income'],]

,days_employed,total_income
12,NaN,NaN
26,NaN,NaN
29,NaN,NaN
41,NaN,NaN
55,NaN,NaN
...,...,...
21489,NaN,NaN
21495,NaN,NaN
21497,NaN,NaN
21502,NaN,NaN


En efecto, los valores ausentes son simétricos.

**Conclusión intermedia**

El número de filas de la tabla filtrada coincide con el número de valores ausentes por lo que la hipótesis de los valores ausentes se trate a que los clientes no tengan trabajo y por ello no demostaron ingresos.

In [9]:
# Vamos a investigar a los clientes que no tienen datos sobre la característica identificada y la columna con los valores ausentes
print('La distribución de valores ausentes de:')
print(data_bank['days_employed'].isna().sum())
# Comprobación de la distribución
dist = data_bank['days_employed'].isna().sum() / data_bank['children'].count()
print(f'La distribución de valores ausentes es:{dist: .1%} del total de clientes.')

La distribución de valores ausentes de:
2174
La distribución de valores ausentes es: 10.1% del total de clientes.


Cerca del 10% de los datos tienen valores ausentes lo que hace inferir que ese 10 % no trabaja ni recibe ingresos.

**Posibles razones por las que hay valores ausentes en los datos**

- La probable causa de los valores ausentes sea que los clientes tengan empleo, es decir, no hayan estado generando ingresos y hayan dejado `days_employed` y `total_income` sin contestar.

In [10]:
# Comprobando la distribución en el conjunto de datos entero
dataframeinfo = {'total': data_bank['income_type'].value_counts(), 'filtered': data_bank_filtered
        ['income_type'].value_counts()}
dataframa = pd.DataFrame(data=dataframeinfo)
dataframa['distribution'] = dataframa['filtered'] / dataframa['total']*100
print(dataframa.head(4))

               total  filtered  distribution
business        5085     508.0      9.990167
civil servant   1459     147.0     10.075394
employee       11119    1105.0      9.937944
entrepreneur       2       1.0     50.000000


In [11]:
income_type_by_ausent = pd.DataFrame(data=data_bank['income_type'].value_counts(ascending=False))
income_type_by_ausent['total_dist'] = income_type_by_ausent/sum(income_type_by_ausent['income_type'])*100
income_type_by_ausent['filtered'] = data_bank_filtered['income_type'].value_counts()
income_type_by_ausent['filtered_dist'] = income_type_by_ausent['filtered']/sum(income_type_by_ausent['filtered'].fillna(0))*100
print(income_type_by_ausent.head(4))

               income_type  total_dist  filtered  filtered_dist
employee             11119   51.656214    1105.0      50.827967
business              5085   23.623693     508.0      23.367065
retiree               3856   17.914053     413.0      18.997240
civil servant         1459    6.778165     147.0       6.761730


**Conclusión intermedia**

Es similar el conjunto de datos original con el conjunto de datos de la distribucion filtrada lo que significa que los valores ausentes **no dependen** del tipo de empleo.

**Conclusiones**

En el análisis de valores ausentes se ha determinado que éstos **podrían** pertenecen a clientes que están desempleados y no tienen un ingreso menusal lo que se correlaciona con las filas que también tienen valores ausentes.

Es una posibilidad la eliminación de los valores ausentes ya que no interfiere en la distribución de las características de los clientes, sin embargo tampoco provocaría un sesgo el mantener y llenar esos valores ausentes.

Un punto importante de notar es que debido a que los clientes no agregaron los datos en la columas de `days_employed`y en `total_income` pero sí hayan llenado su `income_type` diferente de "desempleado" obliga a pensar que no quisieron llenar esos valores.

## Transformación de datos

In [12]:
# Veamos todos los valores en la columna de educación para verificar si será necesario corregir la ortografía y qué habrá que corregir exactamente
print(data_bank.duplicated().sum())

54


In [13]:
# Arreglando registros
data_bank = data_bank.drop_duplicates()

In [14]:
# Comprobando todos los valores en la columna para asegurarnos de que los hayamos corregido
print(data_bank.duplicated().sum())

0


In [15]:
# Veamos la distribución de los valores en la columna `children`
print(data_bank['children'].value_counts())

 0     14107
 1      4809
 2      2052
 3       330
 20       76
-1        47
 4        41
 5         9
Name: children, dtype: int64


In [16]:
dist_child = (data_bank['children'] ==-1).sum()
total_child = data_bank['children'].sum()
dist = dist_child / total_child
print(f'La distribución de clientes que agregaron -1 en la columna de hijos es del:{dist: .1%}.')

La distribución de clientes que agregaron -1 en la columna de hijos es del: 0.4%.


In [17]:
# [arregla los datos según tu decisión]
data_bank['children'] = data_bank['children'].abs()
data_bank['children'] = data_bank['children'].replace(20,2)

In [18]:
# Comprobar la columna `children` de nuevo para asegurarnos de que todo está arreglado
print(data_bank['children'].value_counts())

0    14107
1     4856
2     2128
3      330
4       41
5        9
Name: children, dtype: int64


In [19]:
# Encontrar datos problemáticos en `days_employed`, si existen, y calculamos el porcentaje

negativ = (data_bank['days_employed']<0).sum()
positiv = (data_bank['days_employed']>=0).sum()
percent = negativ / (negativ + positiv)
print(f'El porcentaje de números negativos y flotantes es de:{percent: .1%}.')

El porcentaje de números negativos y flotantes es de: 82.2%.


Como la cantidad de datos problemáticos es significcativa, buscaremos una forma de resolver los valores problemáticos.

In [20]:
# Abordando los valores problemáticos, si existen.
data_bank['days_employed'] = data_bank['days_employed'].abs()
data_bank['days_employed'] = data_bank['days_employed'].fillna(0).astype(int)

In [21]:
int(data_bank['days_employed'].median())

1818

In [22]:
median_days_employed = int(data_bank[data_bank['days_employed']<18250]['days_employed'].median())
median_days_employed

1354

In [23]:
data_bank.loc[data_bank.days_employed>18250,'days_employed'] = median_days_employed

In [24]:
(data_bank['days_employed']>18250).sum()

0

In [25]:
# Comprobando el resultado
n=0
p=0

for data in data_bank['days_employed']:
    if data < 0:
        n = n+1
    else:
        p = p+1
print(f'La cantidad de negativos son: {n:} y la cantidad de positivos: {p:}.')

La cantidad de negativos son: 0 y la cantidad de positivos: 21471.


In [26]:
# Revisar `dob_years` en busca de valores sospechosos y cuenta el porcentaje
print((data_bank['dob_years'] < 18).value_counts())

False    21370
True       101
Name: dob_years, dtype: int64


Se han encontrado clientes donde la edad que marcaron es `0`, lo que hace suponer que o son menores de edad que no registraron su edad o simplemente omitieron escribir su edad, se logra determinar que este grupo de clientes debe ser evaluado con otras de sus características como su estado civil, o si cuentan con un trabajo, ya que eso nos dará mayor claridad de su posible edad. De pasar a que nuestra hipótesis sea correcta podríamos sustituir los valores de `0` por el promedio o la mediana según sea el caso donde no nos provoque un sesgo en nuestro análisis final.

In [27]:
# Resuelviendo los problemas en la columna `dob_years`, si existen
promedio = data_bank['dob_years'].mean()
mediana = data_bank.loc[ data_bank["dob_years"] != 0, 'dob_years' ].median()

data_bank['dob_years'] = data_bank['dob_years'].replace(0,mediana)

In [28]:
# Comprobando el resultado
print((data_bank['dob_years']==0).sum())

0


Limpiaremos los datos de `purpose`:

In [29]:
data_bank['purpose'].unique()

correct_values = ['purchase_house','purchase_car','education','wedding','buy_real_state']
wrong_values = ['purchase of the house', 'car purchase', 'supplementary education',
       'to have a wedding', 'housing transactions', 'education',
       'having a wedding', 'purchase of the house for my family',
       'buy real estate', 'buy commercial real estate',
       'buy residential real estate', 'construction of own property',
       'property', 'building a property', 'buying a second-hand car',
       'buying my own car', 'transactions with commercial real estate',
       'building a real estate', 'housing',
       'transactions with my real estate', 'cars', 'to become educated',
       'second-hand car purchase', 'getting an education', 'car',
       'wedding ceremony', 'to get a supplementary education',
       'purchase of my own house', 'real estate transactions',
       'getting higher education', 'to own a car', 'purchase of a car',
       'profile education', 'university education',
       'buying property for renting out', 'to buy a car',
       'housing renovation', 'going to university']


In [30]:
def replace_values(wrong_values, correct_value):
    for wrong_value in wrong_values:
        data_bank['purpose'] = data_bank['purpose'].replace(duplicates, correct_value)

In [31]:
duplicates = ['purchase of the house','purchase of the house for my family','purchase of my own house',
              'buying property for renting out','housing renovation','housing transactions','housing']
correct_value = 'purchase_house'

replace_values(duplicates, correct_value)

duplicates = ['car purchase','buying a second-hand car','buying my own car','cars','second-hand car purchase',
              'car','to own a car','purchase of a car','to buy a car']
correct_value = 'purchase_car'
replace_values(duplicates, correct_value)


duplicates = ['supplementary education','education','to become educated','getting an education',
              'to get a supplementary education','getting higher education','profile education',
              'university education','going to university']
correct_value = 'education'
replace_values(duplicates, correct_value)

duplicates = ['to have a wedding','having a wedding','wedding ceremony']
correct_value = 'wedding'
replace_values(duplicates, correct_value)

duplicates = ['buy real estate','buy commercial real estate','buy residential real estate','construction of own property',
              'property','building a property','transactions with commercial real estate','building a real estate',
              'transactions with my real estate', 'real estate transactions']
correct_value = 'real_estate'
replace_values(duplicates, correct_value)

In [32]:
print(data_bank['purpose'].unique())

['purchase_house' 'purchase_car' 'education' 'wedding' 'real_estate']


In [33]:
# Veamos los valores de la columna
print(data_bank['family_status'].unique())

['married' 'civil partnership' 'widow / widower' 'divorced' 'unmarried']


In [34]:
print(data_bank['family_status'].value_counts())

married              12344
civil partnership     4163
unmarried             2810
divorced              1195
widow / widower        959
Name: family_status, dtype: int64


Vemos que los datos tienen espacios entre sí, los vamos cambiar con snake_case

In [35]:
# Abordando los valores problemáticos en `family_status`, si existen
data_bank['family_status'] = data_bank['family_status'].replace(
    ['civil partnership', 'widow / widower'],
    ['civil_partnership', 'widow_/_widower'])

In [36]:
# Comprobando el resultado
print(data_bank['family_status'].unique())

['married' 'civil_partnership' 'widow_/_widower' 'divorced' 'unmarried']


In [37]:
# Veamos los valores en la columna
print(data_bank['gender'].value_counts())

F      14189
M       7281
XNA        1
Name: gender, dtype: int64


In [38]:
# Abordando los valores problemáticos, si existen
idx = data_bank[data_bank['gender']=='XNA'].index
data_bank = data_bank.drop(idx)

In [39]:
# Comprobando el resultado
print(data_bank['gender'].value_counts())

F    14189
M     7281
Name: gender, dtype: int64


In [40]:
# Veamos los valores en la columna
print(data_bank['income_type'].value_counts())

employee                       11091
business                        5079
retiree                         3837
civil servant                   1457
unemployed                         2
entrepreneur                       2
student                            1
paternity / maternity leave        1
Name: income_type, dtype: int64


Eliminaremos los valores de carencia estadística.

In [41]:
# Abordando los valores problemáticos, si existen
idx = data_bank[data_bank['income_type'] == 'entrepreneur'].index
data_bank = data_bank.drop(idx)
idx = data_bank[data_bank['income_type'] == 'unemployed'].index
data_bank = data_bank.drop(idx)
idx = data_bank[data_bank['income_type'] == 'student'].index
data_bank = data_bank.drop(idx)
idx = data_bank[data_bank['income_type'] == 'paternity / maternity leave'].index
data_bank = data_bank.drop(idx)

In [42]:
# Comprobando el resultado
print(data_bank['income_type'].value_counts())

employee         11091
business          5079
retiree           3837
civil servant     1457
Name: income_type, dtype: int64


In [43]:
# Comprobando duplicados
print(data_bank.duplicated().value_counts())

False    21256
True       208
dtype: int64


In [44]:
# Comprobando el tamaño del conjunto de datos después de haber ejecutado estas primeras manipulaciones
new_rows = len(data_bank.axes[0])
print(f'Las filas de la tabla original: {rows}.')
print(f'Las filas de la tabla preprocesada: {new_rows}.')
new_data_percent = new_rows / rows
print(f'La cantidad clientes estadísticamente significativos corresponde a un {new_data_percent: .1%} de los clientes totales.')

Las filas de la tabla original: 21525.
Las filas de la tabla preprocesada: 21464.
La cantidad clientes estadísticamente significativos corresponde a un  99.7% de los clientes totales.


# Trabajar con valores ausentes

Vamos a realizar diccionarios para la facilitación de procesos en los valores que vamos a trabajar.

In [45]:
#Haremos los diccionarios.

db_dict_edu = {
    0 : "bachelor's_degree",
    1 : 'secundary_education',
    2 : 'some_college',
    3 : 'primary_education',
    4 : 'graduate_degree'
}

db_dict_family_status = {
    0:'married',
    1:'civil partnership',
    2:'widow_widower',
    3:'divorced',
    4:'unmarried'
    }

#Con los diccionarios, utilizaremos los id's de `family_status` y `education` para limpiar los valores únicos

family_status = []

for family in data_bank['family_status_id']:
    family_status.append(db_dict_family_status.get(family))

db_family = pd.DataFrame(data=family_status)
print('Tabla de valores únicos para `family_status`.')
print(db_family.value_counts())
print('-'*50)

education = []

for edu in data_bank['education_id']:
    education.append(db_dict_edu.get(edu))

db_education = pd.DataFrame(data=education)
print('Tabla para valores únicos de `education`')
print(db_education.value_counts())
        

Tabla de valores únicos para `family_status`.
married              12341
civil partnership     4160
unmarried             2809
divorced              1195
widow_widower          959
dtype: int64
--------------------------------------------------
Tabla para valores únicos de `education`
secundary_education    15186
bachelor's_degree       5247
some_college             743
primary_education        282
graduate_degree            6
dtype: int64


### Restaurar valores ausentes en `total_income`

Como ya se ha mencionado antes, los valores ausentes que hemos encontardo están en las columnas `dob_years` y `total_income`, el plan es abordar esos valores ausentes y evaluar si se deben de eliminar o impotarlos. Para ello primero debemos de de clasificar los valores de ambas columnas, de ahí el siguiente paso será la imputación de datos por medio de estadística descriptiva siendo **mediana** y **media** los probables candidatos.

In [46]:
# Vamos a escribir una función que calcule la categoría de edad
data_bank['dob_years'] = data_bank['dob_years'].astype(int)

def age_group(age):
    if age <= 17:
        return 'children'
    if age <= 30:
        return 'young_men'
    if age <= 40:
        return 'middle_adult'
    if age <= 50:
        return 'adult'
    if age <= 64:
        return 'old_adult'
    return 'retired'

In [47]:
# Probando función
print(age_group(5))
print(age_group(19))
print(age_group(37))
print(age_group(45))
print(age_group(59))
print(age_group(74))

children
young_men
middle_adult
adult
old_adult
retired


In [48]:
# Creando una nueva columna basada en la función
data_bank['age_group'] = data_bank['dob_years'].apply(age_group)

In [49]:
# Comprobar los valores en la nueva columna
print(data_bank['age_group'].value_counts())

old_adult       5756
middle_adult    5730
adult           5364
young_men       3716
retired          898
Name: age_group, dtype: int64


In [50]:
# Creando una tabla sin valores ausentes
data_bank.info()
data_bank_outna = data_bank.dropna(axis=0)
data_bank_outna.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 21464 entries, 0 to 21524
Data columns (total 13 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   children          21464 non-null  int64  
 1   days_employed     21464 non-null  int32  
 2   dob_years         21464 non-null  int32  
 3   education         21464 non-null  object 
 4   education_id      21464 non-null  int64  
 5   family_status     21464 non-null  object 
 6   family_status_id  21464 non-null  int64  
 7   gender            21464 non-null  object 
 8   income_type       21464 non-null  object 
 9   debt              21464 non-null  int64  
 10  total_income      19345 non-null  float64
 11  purpose           21464 non-null  object 
 12  age_group         21464 non-null  object 
dtypes: float64(1), int32(2), int64(4), object(6)
memory usage: 2.1+ MB
<class 'pandas.core.frame.DataFrame'>
Int64Index: 19345 entries, 0 to 21524
Data columns (total 13 columns):
 #   

In [51]:
# Examinando los valores medios y medianos de los ingresos en función de los factores

stat_children = data_bank.groupby('children').agg({'total_income':['mean','median']}).astype(int)
stat_education = data_bank.groupby('education_id').agg({'total_income':['mean','median']}).astype(int)
stat_education = stat_education.rename(db_dict_edu, axis=0)
stat_family_status = data_bank.groupby('family_status').agg({'total_income':['mean','median']}).astype(int)
stat_income_type = data_bank.groupby('income_type').agg({'total_income':['mean','median']}).astype(int)

print(stat_children)
print(stat_education)
print(stat_family_status)
print(stat_income_type)

         total_income       
                 mean median
children                    
0               26418  23026
1               27372  23661
2               27488  23143
3               29322  25155
4               27289  24981
5               27268  29816
                    total_income       
                            mean median
education_id                           
bachelor's_degree          33136  28043
secundary_education        24596  21837
some_college               29040  25608
primary_education          21144  18741
graduate_degree            27960  25161
                  total_income       
                          mean median
family_status                        
civil_partnership        26677  23182
divorced                 27189  23515
married                  27045  23390
unmarried                26938  23149
widow_/_widower          22984  20514
              total_income       
                      mean median
income_type                      
business     

Las categorías que se analizaron son los valores que empíricamente pudieran influir en la imputación de valores ausentes.

In [52]:
# Escribir la función que usaremos para completar los valores ausentes
# Esta función regresa el promedio o la mediana más óptimo y dependiente del valor categórico y subcategórico

dicc_categories = {
    'children' : [0, 1, 2, 3, 4, 5],
    'education' : ["bachelor's_degree", 'secundary_education', 'some_college', 'primary_education','graduate_degree'],
    'family_status' : ['civil_partnership', 'divorced', 'married', 'unmarried', 'widow_/_widower'],
    'income_type' : ['business', 'civil servant', 'employee', 'retiree']
}

def valores_para_imputar(categoria, subcategoria, estadistica):

    
    if categoria == 'children':
        if subcategoria == 0:
            if estadistica == 'promedio':
                return stat_children.loc[0][0]
            return stat_children.loc[0][1]
        if subcategoria == 1:
            if estadistica == 'promedio':
                return stat_children.loc[1][0]
            return stat_children.loc[1][1]
        if subcategoria == 2:
            if estadistica == 'promedio':
                return stat_children.loc[2][0]
            return stat_children.loc[2][1]
        if subcategoria == 3:
            if estadistica == 'promedio':
                return stat_children.loc[3][0]
            return stat_children.loc[3][1]
        if subcategoria == 4:
            if estadistica == 'promedio':
                return stat_children.loc[4][0]
            return stat_children.loc[4][1]
        if subcategoria == 5:
            if estadistica == 'promedio':
                return stat_children.loc[5][0]
            return stat_children.loc[5][1]
    if categoria == 'income_type':
        if subcategoria == 'business':
            if estadistica == 'promedio':
                return stat_children.loc[0][0]
            return stat_children.loc[0][1]
        if subcategoria == 'civil servant':
            if estadistica == 'promedio':
                return stat_children.loc[1][0]
            return stat_children.loc[1][1]
        if subcategoria == 'employee':
            if estadistica == 'promedio':
                return stat_children.loc[2][0]
            return stat_children.loc[2][1]
        if subcategoria == 'retiree':
            if estadistica == 'promedio':
                return stat_children.loc[3][0]
            return stat_children.loc[3][1]
        

In [53]:
# Comprobando si funciona
print(valores_para_imputar('children', 1, 'promedio'))

27372


In [54]:
db_children = data_bank

In [55]:
# Comprobando si tenemos errores
db_children = db_children.fillna(' ')
db_children

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose,age_group
0,1,8437,42,bachelor's degree,0,married,0,F,employee,0,40620.102,purchase_house,adult
1,1,4024,36,secondary education,1,married,0,F,employee,0,17932.802,purchase_car,middle_adult
2,0,5623,33,Secondary Education,1,married,0,M,employee,0,23341.752,purchase_house,middle_adult
3,3,4124,32,secondary education,1,married,0,M,employee,0,42820.568,education,middle_adult
4,0,1354,53,secondary education,1,civil_partnership,1,F,retiree,0,25378.572,wedding,old_adult
...,...,...,...,...,...,...,...,...,...,...,...,...,...
21520,1,4529,43,secondary education,1,civil_partnership,1,F,business,0,35966.698,purchase_house,adult
21521,0,1354,67,secondary education,1,married,0,F,retiree,0,24959.969,purchase_car,retired
21522,1,2113,38,secondary education,1,civil_partnership,1,M,employee,1,14347.61,real_estate,middle_adult
21523,3,3112,38,secondary education,1,married,0,M,employee,1,39054.888,purchase_car,middle_adult


In [56]:
# Reemplazar los valores ausentes si hay algún error
db_children.loc[(db_children['children']==0)&(db_children['total_income']==' '), 'total_income'] = valores_para_imputar('children', 0, 'promedio')
db_children.loc[(db_children['children']==1)&(db_children['total_income']==' '), 'total_income'] = valores_para_imputar('children', 1, 'promedio')
db_children.loc[(db_children['children']==2)&(db_children['total_income']==' '), 'total_income'] = valores_para_imputar('children', 2, 'promedio')
db_children.loc[(db_children['children']==3)&(db_children['total_income']==' '), 'total_income'] = valores_para_imputar('children', 3, 'promedio')
db_children.loc[(db_children['children']==4)&(db_children['total_income']==' '), 'total_income'] = valores_para_imputar('children', 4, 'promedio')
db_children.loc[(db_children['children']==5)&(db_children['total_income']==' '), 'total_income'] = valores_para_imputar('children', 5, 'promedio')
db_children['total_income'] = db_children['total_income'].astype(int)
db_children

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose,age_group
0,1,8437,42,bachelor's degree,0,married,0,F,employee,0,40620,purchase_house,adult
1,1,4024,36,secondary education,1,married,0,F,employee,0,17932,purchase_car,middle_adult
2,0,5623,33,Secondary Education,1,married,0,M,employee,0,23341,purchase_house,middle_adult
3,3,4124,32,secondary education,1,married,0,M,employee,0,42820,education,middle_adult
4,0,1354,53,secondary education,1,civil_partnership,1,F,retiree,0,25378,wedding,old_adult
...,...,...,...,...,...,...,...,...,...,...,...,...,...
21520,1,4529,43,secondary education,1,civil_partnership,1,F,business,0,35966,purchase_house,adult
21521,0,1354,67,secondary education,1,married,0,F,retiree,0,24959,purchase_car,retired
21522,1,2113,38,secondary education,1,civil_partnership,1,M,employee,1,14347,real_estate,middle_adult
21523,3,3112,38,secondary education,1,married,0,M,employee,1,39054,purchase_car,middle_adult


In [57]:
# Comprobar el número de entradas en las columnas
db_children.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 21464 entries, 0 to 21524
Data columns (total 13 columns):
 #   Column            Non-Null Count  Dtype 
---  ------            --------------  ----- 
 0   children          21464 non-null  int64 
 1   days_employed     21464 non-null  int32 
 2   dob_years         21464 non-null  int32 
 3   education         21464 non-null  object
 4   education_id      21464 non-null  int64 
 5   family_status     21464 non-null  object
 6   family_status_id  21464 non-null  int64 
 7   gender            21464 non-null  object
 8   income_type       21464 non-null  object
 9   debt              21464 non-null  int64 
 10  total_income      21464 non-null  int32 
 11  purpose           21464 non-null  object
 12  age_group         21464 non-null  object
dtypes: int32(3), int64(4), object(6)
memory usage: 2.0+ MB


###  Restaurar valores en `days_employed`

Para restaurar los valores de `days_employed` lo que vamos a evaluar son las variables categóricas de `dob_years` y `education`, ya que son las categorías que más influyen en los días trabajados de una persona. Depués de analizar, se ha llegado a la conclusión de que se eligirá la variable categórica `dob_years` ya que entre más edad tenga una persona es más probable que haya trabajado por más días a diferencia de `education` que, si bien influye en los días trabajados de una persona, no es un parámetro tan confiable ya que una persona puede estudiar y trabajar al mismo tiempo.

In [58]:
# Distribución de medias y medianas de `days_employed` en función de los parámetros identificados

#***stat_age_group***  es la media y la mediana de los valores dependientes del grupo de edad.
idx = data_bank[data_bank['days_employed']==0].index
db_days_employed = data_bank.drop(idx, axis=0)
stat_age_group = db_days_employed.groupby('age_group').agg({'days_employed':['mean','median']}).astype(int)
print(stat_age_group['days_employed','mean'])

age_group
adult           2741
middle_adult    2084
old_adult       2384
retired         1733
young_men       1279
Name: (days_employed, mean), dtype: int32


Debido a la **distribución sesgada** que estan reflejando los valores atípicos del grupo de edad con la media y la mediana, se optará por utilizar mejor la mediana antes que la media para impotar los valores de `days_employed`.

In [59]:
# Escribamos una función que calcule medias o medianas (dependiendo de tu decisión) según el parámetro identificado

dic_gpo_edad = {
    'age_group' : ['adult','middle_adult','old_adult','retired','young_men']
}

def imputar_valores_days_employed(gpo_edad, stat):
    if gpo_edad == 'adult':
        if stat == 'promedio':
            return stat_age_group['days_employed','mean'][0]
        return stat_age_group['days_employed','median'][0]
    if gpo_edad == 'middle_adult':
        if stat == 'promedio':
            return stat_age_group['days_employed','mean'][1]
        return stat_age_group['days_employed','median'][1]
    if gpo_edad == 'old_adult':
        if stat == 'promedio':
            return stat_age_group['days_employed','mean'][2]
        return stat_age_group['days_employed','median'][2]
    if gpo_edad == 'retired':
        if stat == 'promedio':
            return stat_age_group['days_employed','mean'][3]
        return stat_age_group['days_employed','median'][3]
    if gpo_edad == 'young_men':
        if stat == 'promedio':
            return stat_age_group['days_employed','mean'][4]
        return stat_age_group['days_employed','median'][4]

In [60]:
# Comprobando la función

age_group = ['adult','middle_adult','old_adult','retired','young_men']

for i in age_group:
    print(imputar_valores_days_employed(i,'promedio'))
    print(imputar_valores_days_employed(i,'mediana'))


2741
1931
2084
1601
2384
1354
1733
1354
1279
1046


In [61]:
db_final = db_children

In [62]:
# Reemplazar valores ausentes con base en el grupo de edad.

age_group = ['adult','middle_adult','old_adult','retired','young_men']

db_final.loc[(db_final['age_group'] == 'adult') & (db_final['days_employed'] == 0), 'days_employed'] = imputar_valores_days_employed('adult', 'mediana')
db_final.loc[(db_final['age_group'] == 'middle_adult') & (db_final['days_employed'] == 0), 'days_employed'] = imputar_valores_days_employed('middle_adult', 'mediana')
db_final.loc[(db_final['age_group'] == 'old_adult') & (db_final['days_employed'] == 0), 'days_employed'] = imputar_valores_days_employed('old_adult', 'mediana')
db_final.loc[(db_final['age_group'] == 'retired') & (db_final['days_employed'] == 0), 'days_employed'] = imputar_valores_days_employed('retired', 'mediana')
db_final.loc[(db_final['age_group'] == 'young_men') & (db_final['days_employed'] == 0), 'days_employed'] = imputar_valores_days_employed('young_men', 'mediana')


In [63]:
# Comprobando las entradas en todas las columnas
db_final.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 21464 entries, 0 to 21524
Data columns (total 13 columns):
 #   Column            Non-Null Count  Dtype 
---  ------            --------------  ----- 
 0   children          21464 non-null  int64 
 1   days_employed     21464 non-null  int32 
 2   dob_years         21464 non-null  int32 
 3   education         21464 non-null  object
 4   education_id      21464 non-null  int64 
 5   family_status     21464 non-null  object
 6   family_status_id  21464 non-null  int64 
 7   gender            21464 non-null  object
 8   income_type       21464 non-null  object
 9   debt              21464 non-null  int64 
 10  total_income      21464 non-null  int32 
 11  purpose           21464 non-null  object
 12  age_group         21464 non-null  object
dtypes: int32(3), int64(4), object(6)
memory usage: 2.6+ MB


## Clasificación de datos

Son dos las categorias donde es necesario categorizar datos, una de ellas es la edad `dob_years` (previo a este punto ya se había optado por una agrupación de edad) y otra categoría es `total_income`, esto es debido a que no podemos agrupar por valores únicos ya que son extensos y es necesario sintetizar los datos para obtener rangos categóricos con efecto estadístico.

In [64]:
# Mostrando los valores de los datos seleccionados para la clasificación
clasification_values = ('debt', 'total_income','income_type','family_status','purpose','children')
db_clasification = db_final.loc[:, clasification_values]
print(db_clasification['children'].value_counts())

0    14102
1     4855
2     2127
3      330
4       41
5        9
Name: children, dtype: int64


In [65]:
# Comprobar los valores únicos
print(db_clasification['total_income'].unique().size)

15386


In [66]:
# Vamos entonces a clasificar los datos de la columna `total_income`:
# Escribamos una función para clasificar los datos en función de temas comunes
print('Valor mínimo de "total_income":')
print(db_clasification['total_income'].min())
print('-'*45)
print('Valor máximo de "total_income":')
print(db_clasification['total_income'].max())
print('-'*45)
print('Promedio de los valores de "total_income":')
print(db_clasification['total_income'].mean())
print('-'*45)
print('Mediana de los valores de "total_income":')
print(db_clasification['total_income'].median())

def total_income_group(income):
    if income <= 10000:
        return '0_to_10k'
    if income <= 20000:
        return '10k_to_20k'
    if income <= 30000:
        return '20k_to_30k'
    if income <= 40000:
        return '30k_to_40k'
    if income <= 50000:
        return '40k_to_50k'
    return 'more_than_50k'

Valor mínimo de "total_income":
3306
---------------------------------------------
Valor máximo de "total_income":
362496
---------------------------------------------
Promedio de los valores de "total_income":
26786.29952478569
---------------------------------------------
Mediana de los valores de "total_income":
24975.5


In [67]:
# Creando una columna con las categorías
db_clasification['total_income_group'] = db_clasification['total_income'].apply(total_income_group)
print(db_clasification['total_income_group'])

0        40k_to_50k
1        10k_to_20k
2        20k_to_30k
3        40k_to_50k
4        20k_to_30k
            ...    
21520    30k_to_40k
21521    20k_to_30k
21522    10k_to_20k
21523    30k_to_40k
21524    10k_to_20k
Name: total_income_group, Length: 21464, dtype: object


In [68]:
# Revisar todos los datos numéricos en la columna seleccionada para la clasificación
print(db_clasification['total_income_group'].value_counts())

20k_to_30k       8181
10k_to_20k       6442
30k_to_40k       3105
40k_to_50k       1492
more_than_50k    1319
0_to_10k          925
Name: total_income_group, dtype: int64


In [69]:
# Obtener estadísticas resumidas para la columna
stat_db_total_income_group = db_clasification.groupby('total_income_group').agg({'total_income':['mean','median','min','max']}).astype(int)
print(stat_db_total_income_group)

                   total_income                      
                           mean median    min     max
total_income_group                                   
0_to_10k                   8101   8376   3306   10000
10k_to_20k                15387  15575  10001   20000
20k_to_30k                25127  26093  20001   29992
30k_to_40k                34303  34064  30009   39998
40k_to_50k                44056  43727  40020   49988
more_than_50k             68618  60287  50012  362496


Los rangos utilizados para clasificar `total_income` se propusieron debido a los datos estadísticos.

## Comprobación de las hipótesis


**¿Existe una correlación entre tener hijos y pagar a tiempo?**

In [70]:
# Comprobando los datos sobre los hijos y los pagos puntuales
db_debt_bychild = db_clasification.groupby(['debt','children']).agg({'children':'count'})
# # Calcular la tasa de incumplimiento en función del número de hijos

def porcentaje(value):
    debt1 = db_debt_bychild['children'][1][value]
    total_child = db_debt_bychild['children'][0][value] + db_debt_bychild['children'][1][value]
    rate = debt1 / total_child *100
    return rate

columns = ['children','rate_debt']
rate = []

for i in range(0,5):
    rate.append(porcentaje(i))

data = {
    'children':[0,1,2,3,4],
    'rate_debt':rate
}
    
rate_children_table = pd.DataFrame(data)
rate_children_table

,children,rate_debt
0,0,7.537938
1,1,9.145211
2,2,9.449929
3,3,8.181818
4,4,9.756098


**Conclusión**

Con base en los resultados anteriores observamos que, el tener hijos aumenta alrededor de **1.5 %** la probabilidad de que el clinete no pague el préstamos llegando a aumentar ligeramente aumentando la cantidad de hijos, solo el tener 3 hijos (8.18%) disminuye un poco la tasa, pero aún sigue estando arriba del 7.5% observado para los que no tienen hijos.

**¿Existe una correlación entre la situación familiar y el pago a tiempo?**

In [71]:
# Comprobando los datos del estado familiar y los pagos a tiempo

ev_deb_family_status = db_clasification.groupby('debt')['family_status'].value_counts()

def porcentaje(value):
    debt1 = ev_deb_family_status[1][value]
    total_family = ev_deb_family_status[0][value] + ev_deb_family_status[1][value]
    rate = debt1 / total_family *100
    return rate

# Calcular la tasa de incumplimiento basada en el estado familiar

columns = ['family_status','rate_debt']
rate = []

for i in range(0,5):
    rate.append(porcentaje(i))

data = {
    'family_status':['married','civil_partnership','unmarried','divorced','widow_/_widower'],
    'rate_debt':rate
}

rate_family_status_table = pd.DataFrame(data)
rate_family_status_table

,family_status,rate_debt
0,married,7.527753
1,civil_partnership,9.326923
2,unmarried,9.754361
3,divorced,7.112971
4,widow_/_widower,6.569343


**Conclusión**

A través del resultado anterior se infiere que personas solteras `unmarried` son el grupo de estado civil que es más probable a NO pagar su deuda, mientras que las personas viudas `widow_/_widower` es más probable que sí paguen un préstamo.

**¿Existe una correlación entre el nivel de ingresos y el pago a tiempo?**

In [72]:
# Comprobando los datos del nivel de ingresos y los pagos a tiempo

ev_deb_income = db_clasification.groupby('debt')['total_income_group'].value_counts()
print(ev_deb_income)
print()
print('-'*50)
# Calcular la tasa de incumplimiento basada en el nivel de ingresos

def porcentaje(value):
    debt1 = ev_deb_income[1][value]
    total_income = ev_deb_income[0][value] + ev_deb_income[1][value]
    rate = debt1 / total_income *100
    return rate

columns = ['total_income_group','rate_debt']
rate = []

for i in range(0,6):
    rate.append(porcentaje(i))

data = {
    'total_income_group' : ['20k_to_30k','10k_to_20k','30k_to_40k','40k_to_50k','more_than_50k','0_to_10k'],
    'rate_debt':rate
}
    
rate_total_income_table = pd.DataFrame(data)
rate_total_income_table

debt  total_income_group
0     20k_to_30k            7484
      10k_to_20k            5892
      30k_to_40k            2863
      40k_to_50k            1390
      more_than_50k         1227
      0_to_10k               869
1     20k_to_30k             697
      10k_to_20k             550
      30k_to_40k             242
      40k_to_50k             102
      more_than_50k           92
      0_to_10k                56
Name: total_income_group, dtype: int64

--------------------------------------------------


,total_income_group,rate_debt
0,20k_to_30k,8.519741
1,10k_to_20k,8.537721
2,30k_to_40k,7.793881
3,40k_to_50k,6.836461
4,more_than_50k,6.974981
5,0_to_10k,6.054054


**Conclusión**

Con base en la tabla anterior se logra observar que el grupo de personas con ingresos entre **10 000** hasta **30 000** mensuales son las más probables de que incumplan con el pago total de la deuda, estos grupos tienen una tasa de incumplimiento de aproximadamente un **8.5 %**. Llama la atención que personas con ingresos menores de 10 000 mensuales tienen la tasa de incumplimiento más baja, es probable que se tenga que hacer una revisión adicional a ello.

**¿Cómo afecta el propósito del crédito a la tasa de incumplimiento?**

In [73]:
# Consultando los porcentajes de tasa de incumplimiento para cada propósito del crédito

ev_deb_purpose = db_clasification.groupby('debt')['purpose'].value_counts()
ev_deb_purpose
# Calcular la tasa de incumplimiento basada en el nivel de ingresos

def porcentaje(value0, value1):
    debt1 = ev_deb_purpose[1][value1]
    total_purpose = ev_deb_purpose[0][value0] + ev_deb_purpose[1][value1]
    rate = debt1 / total_purpose *100
    return rate

rate_real_estate = porcentaje(0,0)
rate_purchase_house = porcentaje(1,3)
rate_purchase_car = porcentaje(2,1)
rate_education = porcentaje(3,2)
rate_wedding = porcentaje(4,4)

rate = [rate_real_estate, rate_purchase_house, rate_purchase_car, rate_education, rate_wedding]
data = {
    'purpose' : ['rate_real_estate', 'rate_purchase_house', 'rate_purchase_car', 'rate_education', 'rate_wedding'],
    'rate_debt':rate
}
    
rate_purpose_table = pd.DataFrame(data)
rate_purpose_table

,purpose,rate_debt
0,rate_real_estate,7.464567
1,rate_purchase_house,6.884952
2,rate_purchase_car,9.333643
3,rate_education,9.217738
4,rate_wedding,7.969152


**Conclusión**

Por último, con base en la tabla anterior se observa que por porcentaje el grupo que más es probable que incumpla con el pago de una deuda es el grupo cuyo propósito de préstamo fue el comprarse un carro (**9.33 %**) y para su educación (**9.22 %**), mientras que para aquellos que hayan solicitado con el objativo de comprar una casa (**6.88 %**) o invertir en bienes raíces (**7.46 %**) es más probable que terminen de pagar el préstamo de manera oportuna.

# Conclusión general 

Se hizo un filtrado y limpieza de datos con los valores ausentes y valores erróneos, como días trabajados en coma flotante y en negativo o en edad cero; se minimizaron los valores únicos al modificar el mismo concepto de valor a formato `snake_case`; se realizó la imputación de datos de ingresos totales y dpias trabajados utilizando la media o mediana según haya sido la mejojr opción; finalmente, se realizó la categorización de grupos en forma de grupos de edad y en grupos dependientes de sus ingresos.

En la comparativa de propósitos las conclusiones son las siguientes:

- El tener hijos aumenta alrededor de **1.5 %** la probabilidad de que el clinete no pague el préstamos.
- Se infiere que personas solteras `unmarried` son el grupo de estado civil que es más probable a NO pagar su deuda, mientras que las personas viudas `widow_/_widower` es más probable que sí paguen un préstamo.
- El grupo de personas con ingresos entre **10 000** hasta **30 000** mensuales son las más probables de que incumplan con el pago total de la deuda, estos grupos tienen una tasa de incumplimiento de aproximadamente un **8.5 %**.
- Existe una relación entre el propósito de préstamo con el incumplimiento de su pago; el grupo que más es probable que incumpla con el pago de una deuda es el grupo cuyo propósito de préstamo fue el comprarse un carro (**9.33 %**) y para su educación (**9.22 %**), mientras que para aquellos que hayan solicitado con el objativo de comprar una casa (**6.88 %**) o invertir en bienes raíces (**7.46 %**) es más probable que terminen de pagar el préstamo de manera oportuna.

Por último, categóricamente hablando los valores más significativos para el incumplimiento de un préstamo en el peor escenario posible sería:

- Cliente con más de 4 hijos.
- Cliente soltero.
- Ingresos mensuales entre 10 000 y 30 000.
- Su propósito es comprar un carro.